In [2]:
import pandas as pd
import numpy as np

In [4]:
def get_clean_values(offers):
    
    clean_values = pd.DataFrame(np.nan,index=range(0,offers.shape[0]),columns=["Currency", "Price", "Surface", "Floor", "Construction year", "Monthly charges"])
    
    # Extract Currency
    is_pln = offers["Price"].str.match(pat=".*zł",case=False,na=False)
    is_eur = offers["Price"].str.match(pat=".*eur",case=False,na=False)
    is_neither = ~(is_pln) & ~(is_eur)

    clean_values.loc[is_pln, "Currency"] = "PLN"
    clean_values.loc[is_eur, "Currency"] = "EUR"
    clean_values.loc[is_neither, "Currency"] = np.nan

    # Extract price, ignore decimal
    clean_values["Price"] = pd.to_numeric(offers["Price"].astype(str).str.replace(" ","").str.extract(pat="(\d+)",expand=False))

    # Extract surface, including decimal
    clean_values["Surface"] = pd.to_numeric(offers["Surface"].astype(str).str.replace(" ","").str.replace(",",".").str.extract(pat="(\d+\.?\d*)",expand=False))

    # Extract floor, floors >10 groupped together with 10
    clean_values["Floor"] = pd.to_numeric(offers["Floor"].astype(str).str.replace("parter","0",case=False).str.replace("suterena","-1",case=False).str.extract(pat="(-?\d+)",expand=False))

    clean_values["Construction year"] = pd.to_numeric(offers["Construction year"].astype(str).str.extract(pat="(\d+)",expand=False))

    # Extract monthly charges, assume all values are in PLN, ignore decimal
    clean_values["Monthly charges"] = pd.to_numeric(offers["Monthly charges"].astype(str).str.replace(" ","").str.extract(pat="(\d+)",expand=False))

    return clean_values


In [5]:
# Load data - excel or SQL database
offers_excel = pd.read_excel("C:/Users/tomasz.starakiewicz/Documents/GitHub/DSBA-python-app/database/scraped_data_rental.xlsx")

In [15]:
# Clean numeric variables and add back to the dataset

offers_clean = get_clean_values(offers_excel)

offers = offers_excel.merge(right=offers_clean,left_index=True,right_index=True,suffixes=(None,"_clean"))

In [14]:
# Feature engineering

offers["Price per sqm"] = offers["Price_clean"] / offers["Surface_clean"]

In [ ]:
# Visualize data